In [12]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [33]:
import numpy as np
import heapq
import importlib
import snapshot_obj
import trace_halo as th
import dataset_compute

In [30]:
importlib.reload(snapshot_obj)
importlib.reload(th)

<module 'trace_halo' from '/home/kassiili/Curvaton_Simulations/analysis/trace_halo.py'>

In [ ]:
%load_ext line_profiler

In [20]:
snap_ref = snapshot_obj.Snapshot("CDM_V1_LR", 127)
snap = snapshot_obj.Snapshot("CDM_V1_LR", 126)

GNs = snap_ref.get_subhalos('GroupNumber')
SGNs = snap_ref.get_subhalos('GroupNumber')

In [31]:
%prun matches_ref = th.match_all(snap_ref,snap,gns=[1])

for idx_ref,match in enumerate(matches_ref):
    print("({},{}) : ({},{})".format(GNs[idx_ref],SGNs[idx_ref],match[0],match[1]))

 (1.0,1.0) : (1.0,0.0)
(1.0,1.0) : (1.0,1.0)
(1.0,1.0) : (-1.0,-1.0)
(1.0,1.0) : (1.0,2.0)
(1.0,1.0) : (1.0,3.0)
(1.0,1.0) : (1.0,4.0)
(1.0,1.0) : (1.0,5.0)
(1.0,1.0) : (1.0,7.0)
(1.0,1.0) : (1.0,8.0)
(1.0,1.0) : (1.0,6.0)
(1.0,1.0) : (1.0,9.0)
(1.0,1.0) : (1.0,10.0)
(1.0,1.0) : (1.0,11.0)
(1.0,1.0) : (1.0,12.0)
(1.0,1.0) : (1.0,14.0)
(1.0,1.0) : (1.0,13.0)
(1.0,1.0) : (1.0,15.0)
(1.0,1.0) : (1.0,16.0)
(1.0,1.0) : (1.0,17.0)
(1.0,1.0) : (1.0,23.0)
(1.0,1.0) : (1.0,20.0)
(1.0,1.0) : (1.0,19.0)
(1.0,1.0) : (1.0,21.0)
(1.0,1.0) : (1.0,22.0)
(1.0,1.0) : (1.0,25.0)
(1.0,1.0) : (-1.0,-1.0)
(1.0,1.0) : (1.0,24.0)
(1.0,1.0) : (1.0,32.0)
(1.0,1.0) : (1.0,30.0)
(1.0,1.0) : (1.0,28.0)
(1.0,1.0) : (1.0,29.0)
(1.0,1.0) : (1.0,31.0)
(1.0,1.0) : (1.0,34.0)
(1.0,1.0) : (1.0,37.0)
(1.0,1.0) : (1.0,36.0)
(1.0,1.0) : (-1.0,-1.0)
(1.0,1.0) : (1.0,42.0)
(1.0,1.0) : (-1.0,-1.0)
(1.0,1.0) : (1.0,39.0)
(1.0,1.0) : (1.0,41.0)
(1.0,1.0) : (1.0,35.0)
(1.0,1.0) : (1.0,43.0)
(1.0,1.0) : (1.0,45.0)
(1.0,1.0) : (1.0

         213716 function calls (211829 primitive calls) in 1.104 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2153    0.270    0.000    0.357    0.000 group.py:253(__getitem__)
     1152    0.204    0.000    0.360    0.000 dataset.py:476(__getitem__)
      735    0.171    0.000    0.171    0.000 {method 'sort' of 'numpy.ndarray' objects}
     1170    0.049    0.000    0.068    0.000 dataset.py:395(__init__)
     4608    0.044    0.000    0.045    0.000 dataset.py:283(shape)
     7202    0.032    0.000    0.032    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        6    0.023    0.004    0.427    0.071 snapshot_obj.py:187(get_subhalo_catalogue)
        2    0.023    0.011    0.396    0.198 snapshot_obj.py:326(get_subhalos_IDs)
        6    0.021    0.003    0.452    0.075 snapshot_obj.py:155(get_subhalos)
     1170    0.017    0.000    0.018    0.000 filters.py:249(get_filters)
9810/7923    0.014    0.000    0.2

In [32]:
%lprun -f snap_ref.get_subhalos_IDs snap_ref.get_subhalos_IDs()

Timer unit: 1e-06 s

Total time: 0.258844 s
File: /home/kassiili/Curvaton_Simulations/analysis/snapshot_obj.py
Function: get_subhalos_IDs at line 326

Line #      Hits         Time  Per Hit   % Time  Line Contents
   326                                               def get_subhalos_IDs(self, fnums=[]):
   327                                                   """ Read IDs of bound particles for each halo.
   328                                                   
   329                                                   Paramaters
   330                                                   ----------
   331                                                   fnums : list of ints, optional
   332                                                       Specifies files, which are to be read
   333                                           
   334                                                   Returns
   335                                                   -------
   336                        

In [35]:
%lprun -f dataset_compute.compute_V1kpc dataset_compute.compute_V1kpc(snap_ref)

Timer unit: 1e-06 s

Total time: 20.1771 s
File: /home/kassiili/Curvaton_Simulations/analysis/dataset_compute.py
Function: compute_V1kpc at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                           def compute_V1kpc(snapshot):
    45                                               """ For each subhalo, calculate the circular velocity at 1kpc. """
    46                                           
    47                                               # Get particle data:
    48         1          4.0      4.0      0.0      part = {}
    49         1     146913.0 146913.0      0.7      part['gns'] = snapshot.get_particles('GroupNumber')
    50         1     142337.0 142337.0      0.7      part['sgns'] = snapshot.get_particles('SubGroupNumber')
    51         1     599099.0 599099.0      3.0      part['coords'] = snapshot.get_particles('Coordinates')\
    52         1      15689.0  15689.0      0.1              * u.cm.to(u.kpc)
  